*****************************************************
# The Social Web Assignment 4: Recommendation

- Instructor: Davide Ceolin, Filip Ilievski, Zubaria Inayat
- TAs: Márton Bodó, Manav Neema, Göktuğ Genckaya and Chiara Michelutti.
- Exercises for Hands-on session 4 
*****************************************************

In this notebook you will use the similarity measures to provide recommendations by comparing users and content based on expressed preferences (ratings). You will also explore textual similarity using a very popular natural language processing library, NLTK. Finally, you will explore recommendations on the Stack Overflow platform.

Required packages:
* feedparser, praw,  nltk

In [ ]:
import sys

!pip install feedparser
!pip install praw
!pip install nltk

> 💡 **Info:** If the you get `command not found: pip` try with `pip3`.

In [ ]:
import sys

!pip3 install feedparser
!pip3 install praw
!pip3 install nltk

In the snippets below, you can find:
* creation of a small toy database in form of a dictionary of dictionaries;
* issuing several similarity measures based on critics' preferences; and
* use those values to obtain meaningful statistics pertaining a user.

# Movie preferences of movie critics
As example data, let us define a python dictionary of movie critics and their ratings of a small set of movies

In [ ]:
critics = {
    'Lisa Rose': {
        'Lady in the Water': 2.5,
        'Snakes on a Plane': 3.5,
        'Just My Luck': 3.0,
        'Superman Returns': 3.5,
        'You, Me and Dupree': 2.5,
        'The Night Listener': 3.0,
    },
    'Gene Seymour': {
        'Lady in the Water': 3.0,
        'Snakes on a Plane': 3.5,
        'Just My Luck': 1.5,
        'Superman Returns': 5.0,
        'The Night Listener': 3.0,
        'You, Me and Dupree': 3.5,
    },
    'Michael Phillips': {
        'Lady in the Water': 2.5,
        'Snakes on a Plane': 3.0,
        'Superman Returns': 3.5,
        'The Night Listener': 4.0,
    },
    'Claudia Puig': {
        'Snakes on a Plane': 3.5,
        'Just My Luck': 3.0,
        'The Night Listener': 4.5,
        'Superman Returns': 4.0,
        'You, Me and Dupree': 2.5,
    },
    'Mick LaSalle': {
        'Lady in the Water': 3.0,
        'Snakes on a Plane': 4.0,
        'Just My Luck': 2.0,
        'Superman Returns': 3.0,
        'The Night Listener': 3.0,
        'You, Me and Dupree': 2.0,
    },
    'Jack Matthews': {
        'Lady in the Water': 3.0,
        'Snakes on a Plane': 4.0,
        'The Night Listener': 3.0,
        'Superman Returns': 5.0,
        'You, Me and Dupree': 3.5,
    },
    'Toby': {'Snakes on a Plane': 4.5, 
             'You, Me and Dupree': 1.0,
             'Superman Returns': 4.0},
}

# **Exercise 1: Finding Similar Users**

In the code below, two different simililarity measures are used: Euclidean distance and the Pearson correlation. If you are not familiar with them, we recommend you look them up to deepen your understanding.

## Euclidian distance

To assess the degree similarity between critics given their respective preferences, we can use the euclidian distance.
Its formula for an N-dimensional space is is:  

$$d(\mathbf{p}, \mathbf{q}) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + \cdots + (p_n - q_n)^2} = \sqrt{\sum_{i=1}^n (p_i - q_i)^2}$$

Because we want a smaller distance to indicate a larger similarity, we will use 1/d(p,q) as our similarity value:

In [ ]:
from math import sqrt

def sim_distance(p1, p2, show_common_dims=False, prefs=critics):
    '''
    Returns a distance-based similarity score between two critics.
    '''

    # Get the list of shared_items
    common_items = []
    for movie in prefs[p1]:
        if movie in prefs[p2]:
            common_items.append(movie)
    # If they have no ratings in common, return 0
    if len(common_items) == 0:
        return 0
    if show_common_dims:
        print("common dimensions between {} and {}: ".format(p1, p2) + str(len(common_items)))
    # Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[p1][movie] - prefs[p2][movie], 2) for movie in common_items])
    
    # return sqrt(sum_of_squares)
    return 1 / sqrt(sum_of_squares)

Using this simple formula, you can calculate a similarity between two critics:

In [ ]:
# get the distance between 'Lisa Rose' and 'Gene Seymour'
sim_distance('Lisa Rose','Gene Seymour') 

For yourself: Try this with other names so you can see who is closer or further.

### Question
Identify at least two potential bugs or errors in the `sim_distance` function code above.

**Answer:** (add your answer here)

A different measure of similarity can be given by pearson correlation.
Which follows:

$$r_{xy} = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}}$$

Where the dividend represents a measure of covariance between dimensions, whereas the divisor is the product of the standard deviation of the scores given by each user.

In [ ]:
def sim_pearson(p1, p2, prefs=critics, verbose=False):
    '''
    Returns the Pearson correlation coefficient for p1 and p2.
    '''

    '''Step 1: Get the list of mutually rated items'''
    common_items = []
    dic = {}
    for movie in prefs[p1]:
        if movie in prefs[p2]:
            common_items.append(movie)
    # If they are no ratings in common, return 0
    if len(common_items) == 0:
        return 0
    '''Step 2: Sum calculations'''
    n_common_items = len(common_items)
    sum1 = sum([prefs[p1][movie] for movie in common_items])
    sum2 = sum([prefs[p2][movie] for movie in common_items])
    # Sums of squares
    sum1Sq = sum([pow(prefs[p1][movie], 2) for movie in common_items])
    sum2Sq = sum([pow(prefs[p2][movie], 2) for movie in common_items])
    # Sum of the products
    pSum = sum([prefs[p1][movie] * prefs[p2][movie] for movie in common_items])
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n_common_items
    den = sqrt((sum1Sq - pow(sum1, 2) / n_common_items) * (sum2Sq - pow(sum2, 2) / n_common_items))
    if den == 0:
        return 0
    r = num / den
    if verbose:
        print("common dimensions: %s" % len(common_items))
        print("Similarity Score for {} and {}: {}".format(p1, p2, r))
    return r

for k in critics.keys():
    sim_pearson('Michael Phillips', k, verbose=True)

Try the examples you used for the eucledian distance again, but now using the pearson correlation:

In [ ]:
sim_pearson('Lisa Rose','Gene Seymour')

### Ranking critics on similarity
The topMatches function below calculates all similarities of a given critic with his peers:

In [ ]:
def topMatches(person, n=5, similarity=sim_pearson, prefs=critics):
    '''
    Returns the best matches for person from the prefs dictionary. 
    Number of results and similarity function are optional params.
    '''
    if similarity not in [sim_distance, sim_pearson]:
        # NB: here we are comparing FUNCTION DEFINITION.
        # We do that only in a jupyter notebook for the sake of simplicity.
        raise ValueError("Callback functions should be: 'sim_pearson' or 'sim_distance'.")
        
    scores = [(similarity(person, other, prefs=prefs), other) for other in prefs
              if other != person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

So you can now get the 3 critics closest to Toby by calling:

In [ ]:
topMatches('Toby',n=3)

*****************************************************
### Task 1.A: Effect of similarity function used
Call the topMatches function on a number of critics with both the default sim_pearson, but also with the sim_distance function. Would you have preference of one over the other? 
*****************************************************

In [ ]:
for critic in critics.keys():
    print(critic)
    print(topMatches(critic,n=3))
    print(topMatches(critic,n=3,similarity=sim_distance))
    print("_____________")

Describe the differences that you found:

**Answer:** (add the answer here)

# **Exercise 2: Recommending Items**

One way to recommend movies to a person would be to rate the movies she has not seen yet by using the scores of the others weighted by the similarity.

In [ ]:
def getRecommendations(person, similarity=sim_pearson, prefs=critics):
    '''
    Gets recommendations for a person by using a weighted average
    of every other user's rankings
    '''
    if similarity not in [sim_distance, sim_pearson]:
        raise ValueError("Callback functions should be: 'sim_pearson' or 'sim_distance'.")

    totals = {}
    simSums = {}
    for other in prefs:
    # Don't compare me to myself
        if other == person:
            continue
        # print(person, other)
        sim = similarity(person, other, prefs=prefs)
    # Ignore scores of zero or lower
        if sim <= 0: 
            continue
        for item in prefs[other]:
            # Only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item] == 0:
                # Similarity * Score
                    totals.setdefault(item, 0)
                    # The final score is calculated by multiplying each item by the
                    #   similarity and adding these products together
                    totals[item] += prefs[other][item] * sim
                    # Sum of similarities
                    simSums.setdefault(item, 0)
                    simSums[item] += sim
    # Create the normalized list
    rankings = [(total / simSums[item], item) for (item, total) in
                totals.items()]
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [ ]:
getRecommendations('Toby', similarity=sim_distance)

In [ ]:
getRecommendations('Toby')

Note that the output does not only consist of a movie title, but also a guess at what the user's rating for each movie would be.
*****************************************************

### Task 2.A: Explainable recommendations
Users often want to know why an item was recommended. Modify the getRecommendations function so that it returns not just the predicted rating and movie title, but also the name of the "closest" critic (the user with the highest similarity score) who watched that movie. You can implement your solution in the function called `getRecommendations_plus`. (see defined bellow)

In [ ]:
# Your Code Here
def getRecommendations_plus():
    '''
    do not forget to add parameters to your function
    '''
    pass

In [ ]:
getRecommendations_plus('Toby')

# **Exercise 3: Transformations** 
**You have been building recommendations based on similar users in Exercise 2, but you could of course also build recommendations based on similar items. In this exercise you will do this.** 

The function is essentially the same, but you need to transfer your data, from:

<code>{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}</code>

to

<code>{'Lady in the Water': {'Lisa Rose': 2.5,'Gene Seymour': 3.0},
'Snakes on a Plane': {'Lisa Rose': 3.5,'Gene Seymour': 3.5}}</code>

This is what the transformPrefs function does. 

You can now create a dictionary for movies with their scores assigned by different people by invoking:

In [ ]:
def transformPrefs(prefs=critics):
    '''
    Transform the recommendations into a mapping where persons are described
    with interest scores for a given title e.g. {title: person} instead of
    {person: title}.
    '''
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # Flip item and person
            result[item][person] = prefs[person][item]
    return result

In [ ]:
movies = transformPrefs()
print(movies)

And find similar items for a particular movie like this:

In [ ]:
topMatches('Superman Returns', prefs=movies)

Or find people who may like a particular movie:

In [ ]:
getRecommendations('Just My Luck', prefs=movies)

*****************************************************
### Task 3.A: Why does the example above work?
Try to follow exactly what is going on in the last call. Notice that Michael and Jack did not rate 'Just my Luck'. How is their rating for it built up?

**Answer:** (add your answer here)

# **Exercise 4: Sentence Similarity**

In [ ]:
# import natural language processing software we need later.
import nltk
from nltk.stem import WordNetLemmatizer


In [ ]:
# Download wordnet and punkt sentence tokenizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('omw-1.4')

Below we have some example sentences to compare later on.

In [ ]:
movies = ["I saw a really good movie last night.",
"The movie is based on the director's life.",
"The movie starts at ten.",
"I took her to a movie.",
"The movie stars Al Pacino.",
"The movie opened last weekend.",
"The movie lasted two hours.",
"He directed several movies.", 
"We just shot another movie.", 
"The movie was set in New York."]

In [ ]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

To get the Jaccard Similarity of two sentences however, we first need to do some preprocessing in the form of Lemmatization and Tokenization.

In [ ]:
def compare(s1, s2):

    #Import a Lemmatizer to get the root form of certain words
    lemmatizer = WordNetLemmatizer() 
    
    #Tokenize both sentences to get each word separately
    word_list1 = nltk.word_tokenize(s1)
    word_list2 = nltk.word_tokenize(s2)
    
#     print("Tokenized sentence", word_list1) #Uncomment to see an example of the tokenized sentence 
    
    #Lemmatize both sentences
    lemmatized_output1 = ' '.join([lemmatizer.lemmatize(w, 'v') for w in word_list1])
    lemmatized_output2 = ' '.join([lemmatizer.lemmatize(w, 'v') for w in word_list2])
    #print(lemmatized_output2)
    return lemmatized_output1, lemmatized_output2

In [ ]:
from nltk.stem import WordNetLemmatizer
for x in range(len(movies)):
    l1, l2 = compare(movies[0], movies[x])
    print("Sentence 1:", l1, '\n', "Sentence 2:", l2, '\n', "Similarity Score:", get_jaccard_sim(l1,l2))

*****************************************************
### Task 4A: In what scenario's could the Jaccard Similarity be more useful than the Euclidean distance and the Pearson Similarity metrics? Why is that? 

**Answer:** (add your answer here)

#  **Exercise 5: Building a Stack Overflow Recommender**

In this exercise, you will build a simple recommender system using **Stack Overflow** data.  
We assume that if a user answers questions associated with a specific **tag** (e.g., `python`), they likely have an interest or expertise in that technology.  

By finding users who are active across similar tags, we can recommend **new technologies** to a target user.

---
####  **Prerequisites**

You will need the **stackapi** library, which provides easy access to the Stack Exchange API without requiring OAuth for publicly available data.

Install it using:

In [ ]:
!pip install stackapi

> 💡 **Info:** If the you get `command not found: pip` try with `pip3`.

In [ ]:
!pip3 install stackapi

#### **Task Description:**
1) **Collection:** Find a list of active users from a specific technology tag (e.g., 'python').
2) **Enrichment:** For every user found, download the other tags they are active in.
3) **Similarity:** Calculate the Jaccard similarity between users based on their shared tags.
4) **Recommendation:** Suggest new tags to a user based on what their "neighbors" are interested in.

-----

#### **Part 1: Collection**. 

We need a "seed" group of users to analyze. To do this, we will ask the API for the top answerers for a specific tag.
- API Endpoint: `tags/{tag}/top-answerers/period`
- **Goal:** Create a dictionary where the keys are Usernames and the values will eventually hold their data.

In [ ]:
from stackapi import StackAPI

# Initialize the API wrapper (we don't need an API key since it will be a small public requests)
SITE = StackAPI('stackoverflow')

def initializeUserDict(tag, count=10):
    user_dict = {}
    print(f"Finding active users in tag: {tag}...")
    
    # Fetch top answerers for the specific tag for the last month
    users = SITE.fetch(f'tags/{tag}/top-answerers/month')
    
    # Slice the list to get only the number of users we want
    top_users = users['items'][:count]
    
    for u in top_users:
        try:
            # We store the user_id (for looking them up later) 
            # and an empty dict for their future tags
            display_name = u['user']['display_name']
            user_id = u['user']['user_id']
            
            user_dict[display_name] = {'user_id': user_id, 'tags': {}} 
        except KeyError:
            pass # Skip users with missing data
            
    return user_dict

Run this cell to collect your initial users.   
Feel free to change `'python'` to `'java'`, `'javascript'`, or `'c++'`

In [ ]:
so_users_raw = initializeUserDict('python', count=10)
print(f"Found {len(so_users_raw)} users.")

To see them we can use:

In [ ]:
for user in so_users_raw:
    print(user)

#### **Part 2: Enrichment**
Knowing a user likes 'python' isn't enough to make recommendations. We need to know what else they are active in. This creates a "User-Item Profile."  

In this step, we iterate through every user found in Part 1 and fetch their personal top tags.  
- API Endpoint: `users/{ids}/tags`
- **Note:** We add a small time delay (time.sleep) to be polite to the server and avoid getting rate-limited. :)

In [ ]:
import time

def fillItems(user_dict, count=20):
    all_items = {}
    
    for username, data in user_dict.items():
        user_id = data['user_id']
        
        try:
            # Fetch the top tags for this specific user
            tags = SITE.fetch(f'users/{user_id}/tags', order='desc', sort='popular')
            
            # Take the top 'count' tags
            user_tags = tags['items'][:count]
            
            for t in user_tags:
                tag_name = t['name']
                
                # We mark the score as 1.0 (indicating the user is active in this tag)
                user_dict[username]['tags'][tag_name] = 1.0
                all_items[tag_name] = 1
                
        except Exception as e:
            print(f"Error fetching for {username}: {e}")
        
        # delay
        time.sleep(0.1) 
        
    # Reformat dictionary to be {User: {Tag: Score, Tag: Score}}
    final_dict = {name: data['tags'] for name, data in user_dict.items()}
    return final_dict

Run this to download the tag data. This may take 10-20 seconds depending on your device.

In [ ]:
prefs = fillItems(so_users_raw, count=15)

`pprint` is a handy library to vislualize data in a more readable format. You can download it in the cell bellow: (for more info see [link](https://docs.python.org/3/library/pprint.html))

In [ ]:
!pip install pprintpp

In [ ]:
from pprint import pprint

In [ ]:
# Let's inspect one user to see what their data looks like
first_user = list(prefs.keys())[0]
pprint(f"Data for {first_user}: {prefs[first_user]}")

#### **Part 3: Similarity (Jaccard Index)**
Now we have our data. To recommend tags, we first need to find which users are similar to each other.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*kd2xV2qX3vvbOiC4XD0xbg.png" width="400">

We will use the **Jaccard Similarity** coefficient.
It measures the overlap between two sets:

$$
J(A, B) = \frac{|A \cap B|}{|A \cup B|}
$$

### Task 5.A: Implementing Jaccard Similarity Function 

If **User A** likes `{Python, Java}` and **User B** likes `{Python, C++}`:

- **Intersection:** `{Python}` (size = 1)  
- **Union:** `{Python, Java, C++}` (size = 3)  
- **Similarity:** $\frac{1}{3}$

In [ ]:
def jaccard_similarity(user1, user2, prefs):
    # Extract the keys (tags) for both users into Sets
    set1 = set(prefs[user1].keys())
    set2 = set(prefs[user2].keys())

    #TODO. IMPLEMENT JACCARD HERE 
    
    # 1. Calculate the intersection (tags shared by both)
    intersection = #TODO

    # 2. Calculate the union (all unique tags across both)
    union = #TODO

    # 3. Handle division by zero (if union is empty, return 0)
    #TODO

    # 4. Return intersection divided by union
    return #TODO

In [ ]:
# Test your function (Should return 0.5)
test_prefs = {'A': {'python':1, 'java':1}, 'B': {'python':1, 'c++':1}}
print(f"Similarity score: {jaccard_similarity('A', 'B', test_prefs)}")

#### **Part 4: Recommendation**

Finally, we generate recommendations.

##### **The Logic**

1. **Find users similar** to the target user.  
2. **Identify tags** that similar users like, but the target user **has not used yet**.  
3. **Weight each tag** by the similarity score:  
   - A tag liked by a *very similar* user receives a **higher weight**.  
   - A tag liked by a *less similar* user receives a **lower weight**.

This produces a ranked list of recommended technologies for the user.

### Task 5.B
Using the logic above, complete the code below to test the recommendations generated for user. 

In [1]:
def getRecommendations(target_user, prefs):
    totals = {}
    simSums = {}

    for other_user in prefs:
        # Don't compare the user to themselves
        if other_user == target_user: 
            continue

        # Get similarity score using your function from Part 3
        sim = #TODO

        # Ignore users with zero or negative similarity
        if sim <= 0: 
            continue

        for tag in prefs[other_user]:
            # Only score tags the target_user DOES NOT already have
            if tag not in prefs[target_user]:
                
                # TODO CALCULATE SCORES
                
                # 1. Weighted Score: Similarity * Item Score (The item score is 1.0)
                totals.setdefault(tag, 0)
                totals[tag] += #TODO
                
                # 2. Keep track of sum of similarities for normalization
                simSums.setdefault(tag, 0)
                #TODO

    # Create the normalized list: Total Score / Sum of Similarities
    rankings = [(total / simSums[tag], tag) for tag, total in totals.items()]

    # Sort descending (best recommendations first)
    #TODO.
    return rankings

SyntaxError: invalid syntax (3653524782.py, line 11)

You can test your code in the cell bellow:

In [ ]:
import random

# Pick a random user
target = random.choice(list(prefs.keys()))
print(f"Recommendations for {target}:")

# Get top 5 recommendations
recs = getRecommendations(target, prefs)
for score, tag in recs[:5]:
    print(f"{tag} (Confidence: {score:.2f})")